# Tutorial: Feed Unit Model with Ideal Property Package


![](feed.PNG)

## Learning Outcomes

- Demonstrate use of the Feed unit model in IDAES


## Problem Statement

In this tutorial, we will be feeding a mixed stream of liquid benzene and liquid toluene. This unit model has no inlets and only one outlet. The outlet conditions are as follows:

Flow Rate = 100 mol/s

Mole Fraction (Benzene) = 0.6

Mole Fraction (Toluene) = 0.4

Temperature = 298.15 K

Pressure = 101325 Pa

For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [1]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, value

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the feed unit model
from idaes.models.unit_models import Feed

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the BTX_ideal property package to create a properties block for the flowsheet
from idaes.models.properties.activity_coeff_models import BTX_activity_coeff_VLE

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Create the ConcreteModel and the FlowsheetBlock objects, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

# Add properties parameter block to the flowsheet with specifications
m.fs.properties = BTX_activity_coeff_VLE.BTXParameterBlock(valid_phase='Liq',
                                                           activity_coeff_model="Ideal")


In [2]:
# Create an instance of the feed unit, attaching it to the flowsheet
# Specify that the property package to be used with the feed is the one we created earlier

m.fs.feed = Feed(property_package=m.fs.properties)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom is: {0}'.format(DOF_initial))

The initial degrees of freedom is: 3


In [3]:
assert DOF_initial == 3

In [4]:
# Fix the outlet conditions

# Feed stream
m.fs.feed.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.feed.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.feed.mole_frac_comp[0, "toluene"].fix(0.4)

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

The final degrees of freedom is: 0


In [5]:
assert DOF_final == 0

### Flowsheet Initialization

In [6]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.feed.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [7]:
# Solve the simulation using ipopt
# Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
result = opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

In [9]:
# Display output report
m.fs.feed.report()


Unit : fs.feed                                                             Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                               Units        Outlet  
    flow_mol                mole / second     100.00
    mole_frac_comp benzene  dimensionless    0.60000
    mole_frac_comp toluene  dimensionless    0.40000
    temperature                    kelvin     298.15
    pressure                       pascal 1.0132e+05


In [10]:
import pytest

# Check results
assert value(m.fs.feed.outlet.flow_mol[0]) == pytest.approx(100, abs=1e-2)
assert value(m.fs.feed.mole_frac_comp[0, "benzene"]) == pytest.approx(0.6, abs=1e-2)
assert value(m.fs.feed.mole_frac_comp[0, "toluene"]) == pytest.approx(0.4, abs=1e-2)